In [51]:
import pandas as pd
import pyparsing as pyp
import os

import string 

os.chdir("/Users/bencampbell/code_louisville/capstone/louisville-bike-accidents")


DATA = "data/raw/cycling_safety_louisville.csv"
df = pd.read_csv(DATA)

In [19]:
# Dropping columns I won't use.
df_dropped = df.drop(['Unnamed: 0', 'COUNTY NAME', 'GPS LATITUDE DECIMAL', 'GPS LONGITUDE DECIMAL',
         'geometry', 'index_right'], axis=1)
#df_dropped

In [28]:
columns = df_dropped.columns
columns

Index(['MASTER FILE NUMBER', 'INVESTIGATING AGENCY', 'LOCAL CODE',
       'COLLISION STATUS CODE', 'ROADWAY NUMBER', 'BLOCK/HOUSE #',
       'ROADWAY NAME', 'ROADWAY SUFFIX', 'ROADWAY DIRECTION CODE',
       'MILEPOINT DERIVED', 'COLLISION DATE', 'COLLISION TIME',
       'INTERSECTION ROADWAY #', 'INTERSECTION ROADWAY NAME',
       'INTERSECTION ROADWAY SFX', 'BETWEEN STREET ROADWAY # 1',
       'BETWEEN STREET ROADWAY NAME 1', 'BETWEEN STREET ROADWAY SFX 1',
       'BETWEEN STREET ROADWAY # 2', 'BETWEEN STREET ROADWAY NAME 2',
       'BETWEEN STREET ROADWAY SFX 2', 'UNITS INVOLVED',
       'MOTOR VEHICLES INVOLVED', 'KILLED', 'INJURED', 'WEATHER CODE',
       'WEATHER', 'ROADWAY CONDITION CODE', 'ROADWAY CONDITION',
       'HIT & RUN INDICATOR', 'ROADWAY TYPE CODE', 'ROADWAY TYPE',
       'DIRECTIONAL ANALYSIS CODE', 'DIRECTIONAL ANALYSIS',
       'MANNER OF COLLISION CODE', 'MANNER OF COLLISION',
       'ROADWAY CHARACTER CODE', 'ROADWAY CHARACTER', 'LIGHT CONDITION CODE',
       'LI

In [63]:
# Renaming some of the columns:

easy_column_renames = ['MASTER FILE NUMBER', 'INVESTIGATING AGENCY', 'LOCAL CODE', 'COLLISION STATUS CODE',
     'ROADWAY NUMBER', 'ROADWAY NAME', 'ROADWAY SUFFIX', 'INTERSECTION ROADWAY NAME',
    'UNITS INVOLVED', 'MOTOR VEHICLES INVOLVED', 'KILLED', 'INJURED', 'WEATHER CODE',
    'WEATHER', 'ROADWAY CONDITION CODE', 'ROADWAY CONDITION', 'ROADWAY TYPE CODE', 'ROADWAY TYPE',
    'DIRECTIONAL ANALYSIS CODE', 'DIRECTIONAL ANALYSIS',
    'MANNER OF COLLISION CODE', 'MANNER OF COLLISION',
    'ROADWAY CHARACTER CODE', 'ROADWAY CHARACTER', 'LIGHT CONDITION CODE',
    'LIGHT CONDITION', 'RAMP FROM ROADWAY ID', 'RAMP TO ROADWAY ID', "Latitude", "Longitude",
    ]

def easy_rename(name:str) -> str:
    return name.replace(" ", "_").lower()

less_easy_column_renames = [
     'INTERSECTION ROADWAY #', 'INTERSECTION ROADWAY SFX', 'BETWEEN STREET ROADWAY # 1',
    'BETWEEN STREET ROADWAY NAME 1', 'BETWEEN STREET ROADWAY SFX 1',
    'BETWEEN STREET ROADWAY # 2', 'BETWEEN STREET ROADWAY NAME 2',
    'BETWEEN STREET ROADWAY SFX 2',]

def less_easy_rename(name:str) -> str:
    name = name.replace("STREET ROADWAY", "street")
    name = name.replace("#", 'number').replace("SFX", 'suffix')
    return easy_rename(name)

misc_column_renames = {
    'BLOCK/HOUSE #': "building_number",
    'ROADWAY DIRECTION CODE': 'roadway_direction',
    'MILEPOINT DERIVED': 'milepoint', 
    'HIT & RUN INDICATOR': "hit_and_run",
    'SECONDARY COLLISION INDICATOR': "secondary_collision"}

renames = {name:easy_rename(name) for name in easy_column_renames}
renames.update({name:less_easy_rename(name) for name in less_easy_column_renames})
renames.update(misc_column_renames)



In [64]:
df_renamed = df_dropped.rename(renames, axis=1)
df_renamed.columns

Index(['master_file_number', 'investigating_agency', 'local_code',
       'collision_status_code', 'roadway_number', 'building_number',
       'roadway_name', 'roadway_suffix', 'roadway_direction', 'milepoint',
       'COLLISION DATE', 'COLLISION TIME', 'intersection_roadway_number',
       'intersection_roadway_name', 'intersection_roadway_suffix',
       'between_street_number_1', 'between_street_name_1',
       'between_street_suffix_1', 'between_street_number_2',
       'between_street_name_2', 'between_street_suffix_2', 'units_involved',
       'motor_vehicles_involved', 'killed', 'injured', 'weather_code',
       'weather', 'roadway_condition_code', 'roadway_condition', 'hit_and_run',
       'roadway_type_code', 'roadway_type', 'directional_analysis_code',
       'directional_analysis', 'manner_of_collision_code',
       'manner_of_collision', 'roadway_character_code', 'roadway_character',
       'light_condition_code', 'light_condition', 'ramp_from_roadway_id',
       'ramp_to_r

In [55]:
list(good_name.scan_string("hello"))

[(ParseResults(['hello'], {}), 0, 5)]